In [1]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from datetime import datetime

scale_percent = 75 # percent of original size

In [2]:
DATA_PATH = r'/scratch/gallowaa/cciw/Data'

In [3]:
# Search for all image files in testing set...
all_images = glob(os.path.join(DATA_PATH,'Videos_and_stills/TestingSet/GLNI/*/*/*/Images/Quad*/*.jpg'))

In [ ]:
#len(all_images)

In [ ]:
#all_images

In [4]:
imagetable_path = os.path.join(DATA_PATH, 'Tables', 'ImageTable.csv')
image_df = pd.read_csv(imagetable_path, index_col=0)

In [5]:
analysis_path = os.path.join(DATA_PATH, 'Tables', 'Analysis.csv')
dive_path = os.path.join(DATA_PATH, 'Tables', 'Dives.csv')

analysis_df = pd.read_csv(analysis_path, index_col=0, dtype={'Count':float})
dive_df = pd.read_csv(dive_path, index_col=0, parse_dates=['Date'])
data_df = pd.merge(analysis_df, dive_df, on='Dive Index', how='outer')
data_df.columns

Index(['Dive Index', 'Quadrat', 'Live Coverage', 'Empty Coverage', 'Biomass',
       'Count', '16mm', '14mm', '12.5mm', '10mm', '8mm', '6.3mm', '4mm', '2mm',
       'Analysis Index', 'Cruise #', 'CSN', 'PSN', 'Date', 'Depth (m)',
       'Overall Coverage', 'Silt (%)', 'Clay (%)', 'Sand (%)', 'Gravel (%)',
       'Cobble (%)', 'Rock (%)', 'Bedrock (%)', 'Boulders (%)', 'Shale (%)',
       'Underlying Substrate Type', 'Underlying Substrate Depth (cm)'],
      dtype='object')

In [6]:
# Number of dives for each year-month
dive_df['Year-Month'] = dive_df['Date'].dt.strftime('%Y-%m')
dive_df_by_year_month = dive_df.groupby(by='Year-Month').count()['PSN']
dive_df_by_year_month.index = [datetime.strptime(year_month, '%Y-%m') for year_month in dive_df_by_year_month.index]
#dive_df_by_year_month

In [7]:
YM = 'Test'

save_path = os.path.join('/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x', YM)
print(save_path)

dive_idx = dive_df[dive_df['Year-Month'].values == YM].index #['Dive Index']
print(dive_idx)

/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Test
Int64Index([], dtype='int64', name='Dive Index')


In [8]:
# analysis indices for data collected in 2018-07
indices = data_df.loc[data_df['Dive Index'].isin(dive_idx)]['Analysis Index'].values

In [9]:
#image_df[image_df['Analysis Index'].isin(indices)]
fnames = image_df[image_df['Analysis Index'].isin(indices)]['Name']

In [10]:
root = os.path.join(DATA_PATH,'Videos_and_stills/TestingSet/GLNI/')

In [11]:
# only load the highest number
#fnames = fnames.to_list()
#print(len(fnames))

In [12]:
#all_images[0].split('/')[-1].split('_')[3].split('-')[1]

In [13]:
#int(t[-1].split('-')[1])

In [14]:
#int('19')

In [16]:
len(all_images)

403

In [15]:
dirs = all_images[0].split('/')[0:-1]

Run this cell twice

In [19]:
# only keep the images with highest suffix number (highest quality)
# run twice
for i, f in enumerate(all_images):
    dirs = f.split('/')
    t = dirs[-1].split('.')[0].split('_')
    img_nb = int(t[-1].split('-')[1])
    
    # check the image number
    if img_nb > 1:
        dirpath = ''
        for d in dirs[0:-1]:
            dirpath += d + '/'
        
        # sometimes numbers are non-contiguous
        while (img_nb - 1) > 0:
            try:
                to_rm = t[0] + '_' + t[1] + '_' + t[2] + '_' + t[3].split('-')[0] + '-' + str(img_nb - 1) + '.jpg'
                all_images.remove(os.path.join(dirpath, to_rm))
            except:
                pass
            img_nb -= 1
print(len(all_images))

174


In [20]:
#os.path.join(dirpath, to_rm)
all_images

['/scratch/gallowaa/cciw/Data/Videos_and_stills/TestingSet/GLNI/3784/2019/Jul.17/Images/Quad2/GLNI_3784-2_2019-07-17_image-3.jpg',
 '/scratch/gallowaa/cciw/Data/Videos_and_stills/TestingSet/GLNI/3784/2019/Jul.17/Images/Quad3/GLNI_3784-3_2019-07-17_image-3.jpg',
 '/scratch/gallowaa/cciw/Data/Videos_and_stills/TestingSet/GLNI/3784/2019/Jul.17/Images/Quad1/GLNI_3784-1_2019-07-17_image-2.jpg',
 '/scratch/gallowaa/cciw/Data/Videos_and_stills/TestingSet/GLNI/3784/2019/Jun.13/Images/Quad2/GLNI_3784-2_2019-06-13_image-3.jpg',
 '/scratch/gallowaa/cciw/Data/Videos_and_stills/TestingSet/GLNI/3784/2019/Jun.13/Images/Quad3/GLNI_3784-3_2019-06-13_image-5.jpg',
 '/scratch/gallowaa/cciw/Data/Videos_and_stills/TestingSet/GLNI/3784/2019/Jun.13/Images/Quad1/GLNI_3784-1_2019-06-13_image-2.jpg',
 '/scratch/gallowaa/cciw/Data/Videos_and_stills/TestingSet/GLNI/503/2019/Jun.19/Images/Quad2/GLNI_503-2_2019-06-19_image-2.jpg',
 '/scratch/gallowaa/cciw/Data/Videos_and_stills/TestingSet/GLNI/503/2019/Jun.19/Image

In [ ]:
month = 'Aug.'
'''
if YM.split('-')[1] == '08':
    month = 'Aug.'
elif YM.split('-')[1] == '07':
    month = 'Jul.'
elif YM.split('-')[1] == '06':
    month = 'Jun.'
else:
    print('invalid')
'''    
print('got month ', month)

In [ ]:
#fnames = ['GLNI_3555-1_2019-08-20_image-1.jpg']

In [21]:
dim = []
for i, f in enumerate(all_images):
    #t = f.split('.')[0].split('_')
    #print(tokens)
    #PSN = t[1].split('-')[0]
    #QUAD = t[1].split('-')[1]
    #file = root + PSN + '/2019/' + month + t[2].split('-')[-1] + '/Images/Quad' + QUAD + '/' + f.split('.')[0] + '.jpg'
    im = cv.imread(f)
    #rgb = cv.cvtColor(im, cv.COLOR_BGR2RGB)
    #plt.imshow(im[::4, ::4, :])
    #plt.title('frame ' + str(i))
    #plt.pause(0.1)
    dim.append((i, im.shape[0]))
dim = np.asarray(dim)

In [22]:
vals, cts = np.unique(dim[:, 1], return_counts=True)
print(vals)
print(cts)
print(cts.sum())

port_mode = dim[:, 0][dim[:, 1] == 7378]
land_mode = dim[:, 0][dim[:, 1] == 4924]

[3788 4924 6738 7378]
[28 62 26 58]
174


In [23]:
28 + 26

54

In [ ]:
len(land_mode)

In [ ]:
100 / (len(port_mode) + len(land_mode))

In [ ]:
portrait = np.asarray(all_images)[port_mode].tolist()
len(portrait)

In [ ]:
land = np.asarray(all_images)[land_mode].tolist()
len(land)

In [ ]:
# files to exclude
exclude_list = ['GLNI_504-3_2019-08-16_image-2.jpg',
                'GLNI_504-1_2019-08-16_image-2.jpg', 
                'GLNI_2908-1_2019-06-11_image-4.jpg',
                'GLNI_12-2_2019-08-14_image-2.jpg',
                'GLNI_12-3_2019-06-04_image-2.jpg',
                'GLNI_2909-3_2019-06-11_image-2.jpg',
                'GLNI_456-3_2019-08-14_image-2.jpg',
                'GLNI_456-1_2019-08-14_image-2.jpg',
                'GLNI_2907-2_2019-06-11_image-2.jpg',
                'GLNI_2907-2_2019-08-21_image-2.jpg', # files already labelled
                'GLNI_12-3_2019-06-18_image-2.jpg',    
                'GLNI_3554-1_2019-08-20_image-2.jpg',
                'GLNI_2907-1_2019-06-11_image-2.jpg',  
                'GLNI_3554-2_2019-06-12_image-2.jpg',
                'GLNI_2907-1_2019-08-21_image-2.jpg',  
                'GLNI_3554-2_2019-08-20_image-2.jpg',
                'GLNI_2907-3_2019-06-11_image-3.jpg',  
                'GLNI_3787-2_2019-06-17_image-1.jpg',
                'GLNI_2908-3_2019-08-21_image-3.jpg',  
                'GLNI_456-2_2019-06-04_image-2.jpg',
                'GLNI_2909-3_2019-06-11_image-4.jpg',  
                'GLNI_501-3_2019-06-05_image-1.jpg',
                'GLNI_2910-1_2019-06-11_image-2.jpg',  
                'GLNI_501-3_2019-07-10_image-1.jpg',
                'GLNI_2910-1_2019-08-21_image-2.jpg',  
                'GLNI_503-2_2019-06-19_image-2.jpg',
                'GLNI_3553-1_2019-07-16_image-5.jpg',  
                'GLNI_503-3_2019-06-05_image-2.jpg',
                'GLNI_3554-1_2019-06-12_image-2.jpg']
print(len(portrait) - len(exclude_list))

In [ ]:
# files already labelled

In [ ]:
#im = cv.imread(portrait[0])
#imc = im[y_start:y_end, x_start:x_end, :]
#plt.imshow(imc)

In [ ]:
len(portrait)

In [ ]:
outfile

In [ ]:
f = portrait[56]
# for portrait mode (7378, 4924)
x_start = 1050 #1250 #775 #450 #1200
y_start = 2150 #2050 #2100
x_end = x_start + 3000
y_end = y_start + 3000
im = cv.imread(f)
imc = im[y_start:y_end, x_start:x_end, :]
width = int(imc.shape[0] * scale_percent / 100)
height = int(imc.shape[1] * scale_percent / 100)
imcr = cv.resize(imc, (width, height)) # resize image
outfile = os.path.join(save_path, f.split('/')[-1].split('.')[0] + '_crop.jpg')
#cv.imwrite(outfile, imcr)
plt.imshow(imcr)

In [ ]:
for i in range(len(portrait)):
    f = portrait[i]
    if f.split('/')[-1] not in exclude_list:
        im = cv.imread(f)
        imc = im[y_start:y_end, x_start:x_end, :]
        width = int(imc.shape[0] * scale_percent / 100)
        height = int(imc.shape[1] * scale_percent / 100)
        imcr = cv.resize(imc, (width, height)) # resize image
        outfile = os.path.join(save_path, f.split('/')[-1].split('.')[0] + '_crop.jpg')
        #cv.imwrite(outfile, imcr)
        plt.imshow(imcr)
        plt.show()
        print(i, "saved ", outfile)

# Landscape mode images

In [ ]:
land_exclude_list = ['GLNI_12-1_2019-08-14_image-2.jpg',
                     'GLNI_12-3_2019-08-14_image-2.jpg',
                     'GLNI_456-1_2019-07-09_image-2.jpg',
                     'GLNI_456-2_2019-07-25_image-1.jpg', 
                     'GLNI_456-2_2019-08-14_image-2.jpg',
                     'GLNI_504-2_2019-08-16_image-2.jpg',
                     'GLNI_2908-2_2019-06-11_image-4.jpg',
                     'GLNI_2910-2_2019-06-11_image-2.jpg', 
                     'GLNI_3783-2_2019-06-13_image-3.jpg',
                     'GLNI_3784-1_2019-06-13_image-2.jpg', 
                     'GLNI_501-2_2019-06-05_image-1.jpg',
                     'GLNI_501-2_2019-06-19_image-2.jpg',
                     'GLNI_12-2_2019-07-09_image-1.jpg', # already labelled by scale  
                     'GLNI_3788-2_2019-06-17_image-3.jpg',
                     'GLNI_456-1_2019-06-04_image-2.jpg']
print(len(land) - len(land_exclude_list))

In [ ]:
#land[i]

In [ ]:
# for land mode (7378, 4924)
x_start = 2150 
y_start = 1050
x_end = x_start + 3000
y_end = y_start + 3000

f = land[24]
im = cv.imread(f)
imc = im[y_start:y_end, x_start:x_end, :]
width = int(imc.shape[0] * scale_percent / 100)
height = int(imc.shape[1] * scale_percent / 100)
imcr = cv.resize(imc, (width, height)) # resize image
outfile = os.path.join(save_path, f.split('/')[-1].split('.')[0] + '_crop.jpg')
cv.imwrite(outfile, imcr)
plt.imshow(imcr)

In [ ]:
for i in range(len(land)):
    f = land[i]
    if f.split('/')[-1] not in land_exclude_list:
        im = cv.imread(f)
        imc = im[y_start:y_end, x_start:x_end, :]
        width = int(imc.shape[0] * scale_percent / 100)
        height = int(imc.shape[1] * scale_percent / 100)
        imcr = cv.resize(imc, (width, height)) # resize image
        outfile = os.path.join(save_path, f.split('/')[-1].split('.')[0] + '_crop.jpg')
        #cv.imwrite(outfile, imcr)
        plt.imshow(imcr)
        plt.show()
        print(i, "saved ", outfile)

In [ ]:
ct = 0

for i in range(len(land)):
    f = land[i]
    if f.split('/')[-1] not in land_exclude_list:
        im = cv.imread(f)
        imc = im[y_start:y_end, x_start:x_end, :]
        width = int(imc.shape[0] * scale_percent / 100)
        height = int(imc.shape[1] * scale_percent / 100)
        imcr = cv.resize(imc, (width, height)) # resize image
        outfile = os.path.join(
            save_path, 'land/' + f.split('/')[-1].split('.')[0] + '_crop.jpg')
        cv.imwrite(outfile, imcr)
        print(ct, "saved ", outfile)
        ct += 1

In [ ]:
#land[29:]

In [ ]:
#land[43]
ct=0
f = land[29]
if f.split('/')[-1] not in land_exclude_list:
    im = cv.imread(f)
    imc = im[y_start:y_end, x_start:x_end, :]
    width = int(imc.shape[0] * scale_percent / 100)
    height = int(imc.shape[1] * scale_percent / 100)
    imcr = cv.resize(imc, (width, height)) # resize image
    outfile = os.path.join(
        save_path, 'land/good/' + f.split('/')[-1].split('.')[0] + '_crop.jpg')
    cv.imwrite(outfile, imcr)
    print(ct, "saved ", outfile)
    ct += 1

In [ ]:
outfile = os.path.join(
        save_path, 'land/' + f.split('/')[-1].split('.')[0] + '_crop.jpg')
#cv.imwrite(outfile, imcr)

In [ ]:
x_start = 2150 
y_start = 800
x_end = x_start + 3000
y_end = y_start + 3000
print(x_end - x_start)
print(y_end - y_start)
imc = im[y_start:y_end, x_start:x_end, :]
plt.figure(figsize=(6, 12))
plt.imshow(imc)

In [ ]:
width = int(imc.shape[0] * scale_percent / 100)
height = int(imc.shape[1] * scale_percent / 100)
imcr = cv.resize(imc, (width, height)) # resize image
cv.imwrite(os.path.join(save_path, f.split('/')[-1].split('.')[0] + '_crop.jpg'), imcr)

In [ ]:
"""
# for landscape mode (4924, 7378)
x_start, x_end = 2100, 5100
#x_start, x_end = 2200, 5200
#y_start, y_end = 800, 3800
y_start, y_end = 1000, 4000
"""

In [ ]:
#%matplotlib inline
#i = 2
#root_fname = fnames.values[portrait_mode][i].split('/')[-1].split('.')[0]
#guid = image_df[image_df['Name'].str.contains(root_fname)]['Analysis Index'].astype('int64')
#data_df[data_df['Analysis Index'].values == guid.values]

guid = image_df[image_df['Name'].str.contains('3801-1_2018-08')]['Analysis Index'].astype('int64')
data_df[data_df['Analysis Index'].values == np.unique(guid.values)]